In [ ]:
#!pip install python-binance -qq
#!pip install -qq wandb
#!conda install -c conda-forge sktime-all-extras -y
#!pip install sktime
#!pip install -qq --upgrade tsfresh
#!pip install scipy==1.5

In [ ]:
#!pip install --upgrade protobuf

In [ ]:
!nvidia-smi --query-gpu=gpu_name,driver_version,memory.total --format=csv

name, driver_version, memory.total [MiB]
NVIDIA GeForce GTX 1050 Ti, 470.103.01, 4036 MiB


In [ ]:
#%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


## **Preliminaries** 

In [ ]:
import math
import numpy as np
#from numba import cuda
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import norm
from matplotlib.pyplot import figure
import statistics

from keras.layers import Dropout
from keras.callbacks import ModelCheckpoint,ReduceLROnPlateau
#from tqdm.notebook import tqdm

#Importing the Libraries
%matplotlib inline
import matplotlib. pyplot as plt
import matplotlib
from sklearn.preprocessing import MinMaxScaler
from keras.layers import LSTM, Dense, Dropout
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.dates as mandates
from sklearn.preprocessing import MinMaxScaler
from sklearn import linear_model
from keras.models import Sequential
from keras.layers import Dense
import keras.backend as K
from keras.callbacks import EarlyStopping
#from keras.optimizers import adam_v2, adamax_v2, adadelta_v2, adagrad_v2, rmsprop_v2
from keras.optimizers import Adam, Adamax, Adadelta, Adagrad
from keras.models import load_model
from keras.layers import LSTM, GRU, Flatten,BatchNormalization
from tensorflow.keras import regularizers
from keras.utils.vis_utils import plot_model
import keras
import pickle

#import tsfresh
#from tsfresh import extract_features, extract_relevant_features, select_features
#from tsfresh.utilities.dataframe_functions import impute
#from tsfresh.feature_extraction import ComprehensiveFCParameters
#from tsfresh.feature_extraction.settings import Feature_Extraction_Settings
#from sktime.transformers.series_as_features.summarize import TSFreshFeatureExtractor

from sklearn.metrics import classification_report, recall_score, precision_score

#from statsmodels.tsa.stattools import grangercausalitytests
from sklearn.metrics import classification_report
from datetime import datetime

from IPython.display import clear_output 
from scipy import signal
from sklearn.metrics import classification_report, recall_score, precision_score

from sktime.forecasting.model_selection import temporal_train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA
from tensorflow.keras.callbacks import LearningRateScheduler

import wandb
#from wandb.keras import WandbCallback

from IPython.display import clear_output
from sklearn.decomposition import PCA
#from keras_tqdm import TQDMNotebookCallback

In [ ]:
root = '/content/drive/MyDrive/'
path = root+'generate_dataset/movement/'

In [ ]:
#path = '/home/kurniawan/jupyter/'

In [ ]:
def dataset_timesteps(dataset, look_back=0):
    dataset = dataset.tolist()
    dataset_final = []
    temp_list = []
    temp_list.append(dataset)
    
    for lb in range( look_back ):
        temp = [ 0 for x in range(lb+1) ]
        temp_list.append( temp + dataset )
    
    for idx in range( len(dataset) ):
        temp = []
        for ls in range( len(temp_list) ):
            temp.append( temp_list[ls][idx] )
        dataset_final.append(temp)
         
    return np.array( dataset_final[look_back:] )

In [ ]:
def read_dataset(target, features, nPca, look_back, sqr, train, tes):

    df = pd.read_csv(path+"dataset/ETHBUSD-15m-data.csv", parse_dates=['timestamp'])
    del df['close_time']
    del df['quote_av']
    del df['tb_base_av']
    del df['tb_quote_av']
    del df['ignore']

    # target
    df['target'] = df[target].shift(-1)

    # volume / trades
    vt_mult = 1
    df['vt'] = ( df['volume']/df['trades'] ) * vt_mult
    #del df['trades']
    #del df['volume']

    # precentage change
    pct_mult = 2500
    df['pct_change'] = df['close'].pct_change(1)
    df['target_pct_change'] = df['target'].pct_change(1)
    df.dropna(inplace=True)

    # movement
    fee = 0.0016
    move_mult = 1000
    df['move'] = df['pct_change'].apply(lambda x: 1 if x > fee else 0)
    df['target_move'] = df['target_pct_change'].apply(lambda x: 1 if x > fee else 0)
    df['move'] = df['move'] * move_mult

    # profit
    df['profit'] = df['pct_change'] - fee

    # squared
    df['open'] = df['open']
    df['high'] = df['high']
    df['low'] = df['low']
    df['close'] = df['close']

    df['vt'] = df['vt'] 
    df['pct_change'] = df['pct_change']
    df['move'] = df['move']
    df['profit'] = df['profit']

    # slice
    timeframe_day = int(( 60 / 15 ) * 24)
    timeframe = int( timeframe_day * train) +2 +look_back + int( timeframe_day * tes) 
    df = df.iloc[-(timeframe): -2]
    df.dropna(inplace=True)
    df.reset_index(inplace=True)
    del df['index']

    pca = PCA(n_components=nPca)
    principalComponents = pca.fit_transform(df[features])
    #df['pca'] = principalComponents

    if nPca != 0:
      feat = pd.DataFrame( principalComponents )
    else:
      feat = pd.DataFrame( principalComponents )
    
    feat = feat ** sqr
    
    from sklearn import preprocessing

    x = feat.values #returns a numpy array
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    feat = pd.DataFrame(x_scaled)

    #return df

    y_train, y_test, x_train, x_test = temporal_train_test_split(y = (df['target'].values),
                                                              X = ( df[features].values ),
                                                              test_size = (timeframe_day * tes) +1 ) 


    x_train = dataset_timesteps(x_train, look_back=look_back)
    x_test = dataset_timesteps(x_test, look_back=look_back)


    y_train = y_train[look_back:]
    y_test = y_test[look_back:]

    x_train = np.array(x_train.tolist())
    x_test = np.array(x_test.tolist())


    #x_train = df[features].values[:-(timeframe_day * 7)]
    #y_train = df['target'].values[:-(timeframe_day * 7)]

    #x_test = df[features].values[-(timeframe_day * 7):]
    #y_test = df['target'].values[-(timeframe_day * 7):]

    # reshape input to be [samples, time steps, features]
    x_train = np.reshape(x_train, (x_train.shape[0], (look_back+1), len(features)))
    x_test = np.reshape(x_test, (x_test.shape[0], (look_back+1), len(features) ))

    
    return x_train, x_test, y_train, y_test

In [ ]:
def init_model(x_train, mode, nodes, bias, dropout, flatten, activation, regularizer, layer):
  #nodes = 80
  #bias = True
  #nodes = config.nodes

  model = Sequential()

  if mode == 'GRU':
      model.add(GRU(nodes, input_shape=(x_train.shape[1], x_train.shape[2]), use_bias=bias, activation = activation, 
                    return_sequences=True,  activity_regularizer=regularizer))  
      if dropout != 0: model.add(Dropout(dropout))
      for lay in range(layer):
          model.add(GRU(int(nodes), use_bias=bias, activation = activation, return_sequences=True, activity_regularizer=regularizer))
          if dropout != 0: model.add(Dropout(dropout))

    
  elif mode == 'LSTM':
      model.add(LSTM(nodes, input_shape=(x_train.shape[1], x_train.shape[2]), use_bias=bias, activation = activation, 
                    return_sequences=True,  activity_regularizer=regularizer))
      if dropout != 0: model.add(Dropout(dropout))
      for lay in range(layer):
          model.add(LSTM(int(nodes), use_bias=bias, activation = activation, return_sequences=True, activity_regularizer=regularizer))
          if dropout != 0: model.add(Dropout(dropout))
      

  if flatten == True:
    model.add(Flatten())
    if dropout != 0: model.add(Dropout(dropout))
    
  model.add(Dense(int(nodes),  use_bias=bias, activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(int(nodes*0.8),  use_bias=bias, activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(int(nodes*0.6),  use_bias=bias, activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(int(nodes*0.4),  use_bias=bias, activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(int(nodes*0.2),  activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(int(nodes*0.1),  activation = activation, activity_regularizer=regularizer))
  if dropout != 0: model.add(Dropout(dropout))
  model.add(Dense(1, use_bias=bias, activation = activation, activity_regularizer=regularizer))

  return model

In [ ]:
def get_profit(y, y_pred, pct_profit):
  fee = 0.0016
  eval = pd.DataFrame(y, columns = ['actual'])
  eval['pred'] = y_pred
  #eval['pred'] = eval['pred'].shift(-1)
  #eval.dropna(inplace=True)
  #print(len(y), len(y_pred))

  # get moe
  moe = 0 
  for i, row in eval.iterrows():
    moe += row['actual'] - row['pred']
  moe = moe/len(eval)
  eval['pred_moe'] = eval['pred'] + moe

  # get percentage change
  eval['actual_pct_change'] = eval['actual'].pct_change(1)
  eval['pred_pct_change'] = eval['pred'].pct_change(1)
  eval['pred_moe_pct_change'] = eval['pred_moe'].pct_change(1)
  eval.dropna(inplace=True)
  #print(eval[:5])

  # get movement
  eval['actual_move'] = eval['actual_pct_change'].apply(lambda x: 1 if x>fee else 0)
  eval['pred_move'] = eval['pred_pct_change'].apply(lambda x: 1 if x>fee else 0)
  eval['pred_moe_move'] = eval['pred_moe_pct_change'].apply(lambda x: 1 if x>fee else 0)

  # percentage change - fee
  eval['actual_profit'] = eval['actual_pct_change'].apply(lambda x: x - fee)
  eval['pred_profit'] = eval['pred_pct_change'].apply(lambda x: x - fee)
  eval['pred_moe_profit'] = eval['pred_moe_pct_change'].apply(lambda x: x - fee)

  # sum profit
  profit = eval[eval[pct_profit]==1]
  init_asset = 100
  asset = 0
  for i, row in profit.iterrows():
    asset += init_asset * row['actual_profit']
  print(profit[:3])

  profit = eval[eval['actual_move']==1]
  init_asset = 100
  actual_asset = 0
  for i, row in profit.iterrows():
    actual_asset += init_asset * row['actual_profit']

  # diff
  diff = 0
  for i, row in eval.iterrows():
    if row['actual_move'] == row['pred_move']:
      diff += 1
    else:
      diff += 0

  actual_diff = len(eval)

  # recall
  try:
    recall = recall_score(eval['actual_move'], eval['pred_move'], pos_label = 1 )
  except:
    recall = 0

  plt.plot(eval['actual'], label='actual')
  plt.plot(eval['pred_moe'], label='pred_moe')
  plt.legend(loc='best')
  plt.show()
    
  if (asset/init_asset)*100 != 0 and (asset/init_asset)*100 > 0:
    for i, row in eval.iterrows():
        wandb.log( { 'actual': row['actual'],
                   'pred_moe': row['pred_moe'],
                   'index_plt': i
                   } )

  return (actual_asset/init_asset)*100, (asset/init_asset)*100, recall, moe, actual_diff, diff

In [ ]:
def init_callbacks(initial_lr, epochs, lr_decay, monitor):
  from tqdm.keras import TqdmCallback

  filepath = path+"weight/"+str(wandb.run.name)+"-weight.hdf5"
  checkpoint = ModelCheckpoint(filepath, monitor=monitor, verbose=0, save_best_only=True, mode='auto')

  if lr_decay != 0:
    initial_learning_rate = initial_lr
    epochs = epochs
    decay = initial_learning_rate / (epochs/lr_decay)
    def lr_time_based_decay(epoch, lr):
      return lr * 1 / (1 + decay * epoch)
    callbacks_list = [checkpoint,
                    LearningRateScheduler(lr_time_based_decay, verbose=0),
                    TqdmCallback()
                   ] 
  else:
    callbacks_list = [checkpoint,
                    TqdmCallback()
                   ] 

  return callbacks_list

In [ ]:
def train(retry = 0): 
  
  # wandb
  hyperparams = dict(
    nodes=80,
    lr=0.001      
    )
  wandb.init(project="my-test-project", entity="codechrl", config=hyperparams)
  config = wandb.config
  
  # dataset
  features = ['open']
  target = ['close']

  features = config.features

  while True:
    #df = read_dataset(target, features)
    x_train, x_test, y_train, y_test = read_dataset(target, features, config.pca, config.look_back, config.sqr, config.train, config.tes)
    #print(x_train[:1])
    if len(x_train) != 0: break
  

  # init model
  tf.get_logger().setLevel('ERROR')
  model = init_model(x_train, config.mode, config.nodes, 
                     config.bias, config.dropout, config.flatten, 
                     config.activation, config.regularizer,
                    config.layer)

  #init_weights = model.get_weights()

  # callbacks
  callbacks_list = init_callbacks(config.initial_lr, 
                                  config.epochs, 
                                  config.lr_decay,
                                  config.monitor)

  # compile model
  
  model.compile(loss = config.loss_func, optimizer = config.optimizer)

  # fit

  #try:
  model.fit(x_train, y_train,
              validation_data=(x_test, y_test),
              epochs = config.epochs, 
              batch_size = config.batch_size,
              verbose = 0, 
              shuffle = False,
              callbacks = callbacks_list,)

    #train(retry=retry)
    
  # eval
  # load model
  filepath = path+"weight/"+str(wandb.run.name)+"-weight.hdf5"
  model = load_model(filepath)

  # predict
  y_pred = model.predict(x_test)
  print( 'train ', x_train[:2] )
  #print('test ', y_test[:2])
  #print('pred ',y_pred[:2])
  y_pred = [x[0] for x in y_pred]
  print('pred ',y_pred[:2])

  # profitability
  actual_profit, profit, recall, moe, actual_diff, diff = get_profit(y_test, y_pred, config.pct_profit)
  difference = ( diff / actual_diff if actual_diff!=0 else 0 )

  print(actual_profit, profit)
     
  if profit <= 0 and retry!=2:
    print('RETRY')
    train(retry=retry+1)
    del model, callbacks_list, filepath, y_pred, 
    actual_profit, profit, recall, moe, 
    actual_diff, diff, difference,
    x_train, x_test, y_train, y_test 
    
  elif profit <= 0 and retry==2:
    #clear_output()
    # log into wandb  
    pr = (-100/(profit-0.0001))/10000000
    wandb.log( { 'retry': retry+1,
              'profit': profit,
              'moe': abs(moe),
              'recall': recall,
              'diff': difference,
              #'init_weights': init_weights,
              'hm': statistics.harmonic_mean( [ pr, recall, difference ] )
              } )

  else:
    # log into wandb  
    wandb.log( { 'retry': retry+1,
              'profit': profit,
              'moe': abs(moe),
              'recall': recall,
              'diff': difference,
              #'init_weights': init_weights,
              'hm': statistics.harmonic_mean( [ profit/actual_profit, recall, difference ] )
              } )

  #if int(str(wandb.run.name)[-1]) % 5 == 0: 
  clear_output()
    
  #if profit > 0:
  #    del model, callbacks_list, filepath, y_pred, actual_profit, profit, recall, moe, actual_diff, diff, difference, init_weights

In [ ]:
import itertools
list_features = []
stuff =  ['open', 'high', 'low', 'close', 'trades', 'volume', 'vt', 'pct_change', 'move' , 'profit']
for L in range(0, len(stuff)+1):
    for subset in itertools.combinations(stuff, L):
        list_features.append( list(subset) )

list_features = list_features[1:]
#list_features

In [ ]:
sweep_config = {
    'method': 'bayes',         
    'metric': {
        'name': 'hm',     
        'goal': 'maximize'      
    },
    'parameters': {
        
        'mode': { 
          'values' : ['LSTM', 'GRU']
        },
        
        'dropout': { 
          'values' : [0.0001,  0.01, 0.1, 0.15, 0.2, 0.5]
        },
        
        'flatten': { 
          'values' : [True]
        },
        
        'regularizer': { 
          'values' : ['l1', 'l2', 'l1_l2']
        },
        
        'look_back': { 
          'values' : [0,1,2,3,4,8,16, 24]
        },
        
        'bias': { 
          'values' : [True, False]
        },
        
        'activation': { 
          'values' : ['elu', 'relu']
        },
        
        'initial_lr': {
          'values' : [0.1, 0.01, 0.001, 0.0001]
        },

        'epochs': {
          'values' : [50, 100, 500, 1000]
        },

        'lr_decay': {
          'values' : [0, 0.0001, 0.001, 0.01, 0.1, 1, 5, 10] 
        },

        'loss_func': {
          'values' : ['mean_squared_error', 'mean_squared_error', 'huber_loss', 'mean_absolute_percentage', 'mean_squared_logarithmic', 'cosine_similarity', 'log_cosh' ]
        },

        'optimizer': {
          'values' : [ 'adam', 'adagrad', 'nadam', 'sgd', 'rmsprop', 'adadelta', 'ftrl', 'adamax'  ]
        },

        'batch_size': {
          'values' : [ int(96*7*0.25) ]
        },

        'pca': {
          'values' : [ 1 ]
        }, 

        'monitor': {
          'values' : [ 'loss', 'val_loss' ]
        },

        'layer': {
            'values': [ 0,1,2,3 ] 
        },

        'sqr': {
            'values': [ 1, 2 ] 
        },

        'nodes': {
            'values': [ 250 ] 
        },

        'pct_profit': {
            'values': [ 'pred_move', 'pred_moe_move' ] 
        },

        'features': {
            'values':  list_features
        },

        'train': {
            'value': 7*1
        },

        'tes': {
            'value':  1 
        }

    }
}

In [ ]:
import os
os.environ["WANDB_DISABLE_CODE"] = "True"
os.environ["WANDB_AGENT_MAX_INITIAL_FAILURES"] = '1000'

In [ ]:
# b5cfb2c82b057d3f137a7c529534bd0997ae4d3c
sweep_id = wandb.sweep(sweep_config, project="my-sweep-test-project", entity="codechrl")

Create sweep with ID: z3teakea
Sweep URL: https://wandb.ai/codechrl/my-sweep-test-project/sweeps/z3teakea


In [ ]:
#sweep_id = 'igvj5e7z'
wandb.agent(sweep_id, function = train)

wandb: Agent Starting Run: xsw62e6j with config:
wandb: 	activation: elu
wandb: 	batch_size: 168
wandb: 	bias: False
wandb: 	dropout: 0.1
wandb: 	epochs: 100
wandb: 	features: ['low', 'trades', 'volume', 'move']
wandb: 	flatten: True
wandb: 	initial_lr: 0.001
wandb: 	layer: 0
wandb: 	look_back: 4
wandb: 	loss_func: huber_loss
wandb: 	lr_decay: 0.0001
wandb: 	mode: GRU
wandb: 	monitor: loss
wandb: 	nodes: 250
wandb: 	optimizer: adadelta
wandb: 	pca: 1
wandb: 	pct_profit: pred_move
wandb: 	regularizer: l1
wandb: 	sqr: 2
wandb: 	tes: 1
wandb: 	train: 7
wandb: Currently logged in as: codechrl (use `wandb login --relogin` to force relogin)
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.
/home/kurniawan/anaconda3/envs/keras_gpu/lib/python3.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPytho

Run xsw62e6j errored: NotImplementedError("Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported")
wandb: ERROR Run xsw62e6j errored: NotImplementedError("Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported")
wandb: Agent Starting Run: nxjkk2fg with config:
wandb: 	activation: elu
wandb: 	batch_size: 168
wandb: 	bias: False
wandb: 	dropout: 0.0001
wandb: 	epochs: 100
wandb: 	features: ['open', 'high', 'pct_change', 'profit']
wandb: 	flatten: True
wandb: 	initial_lr: 0.01
wandb: 	layer: 1
wandb: 	look_back: 2
wandb: 	loss_func: mean_absolute_percentage
wandb: 	lr_decay: 1
wandb: 	mode: GRU
wandb: 	monitor: loss
wandb: 	nodes: 250
wandb: 	optimizer: adadelta
wandb: 	pca: 1
wandb: 	pct_profit: pred_moe_move
wandb: 	regularizer: l2
wandb: 	sqr: 1
wandb: 

Run nxjkk2fg errored: NotImplementedError("Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported")
wandb: ERROR Run nxjkk2fg errored: NotImplementedError("Cannot convert a symbolic Tensor (gru/strided_slice:0) to a numpy array. This error may indicate that you're trying to pass a Tensor to a NumPy call, which is not supported")
wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
wandb.finish()